# Open Route Service Execution

### Closest Facility

In [ ]:
ishp = '/mnt/e/rms/madeira/rms_bgri11_pnt01.shp'

fshp = '/mnt/e/rms/equipasocial/creches_mad.shp'

oshp = '/mnt/e/rms/acc_pop21/acc_creches_mad01.shp'

In [ ]:
import pandas as pd

from glass.rd.shp import shp_to_obj
from glass.wt.shp import df_to_shp
from glass.prj.obj import df_prj

from glass.mob.ors import matrix_od

In [ ]:
idf = shp_to_obj(ishp)
fdf = shp_to_obj(fshp)

_idf = df_prj(idf, 4326)
_fdf = df_prj(fdf, 4326)

In [ ]:
i_coords = [[x,y] for x,y in zip(_idf['geometry'].x , _idf['geometry'].y)]
f_coords = [[x,y] for x,y in zip(_fdf['geometry'].x , _fdf['geometry'].y)]

In [ ]:
i_idx = list(range(len(i_coords)))
f_idx = [len(i_idx) + i for i in range(len(f_coords))]

locations = i_coords + f_coords

In [ ]:
r = matrix_od(
    locations, idx_src=i_idx,
    idx_dest=f_idx, impedance='foot-walking'
)

In [ ]:
jd = r["json"]
mtx = jd["durations"]

cols = [f'loc_{str(i)}' for i in range(len(f_coords))]

mtx_df = pd.DataFrame(mtx, columns=cols)

In [ ]:
# Convert to minutes
for c in cols:
    mtx_df[c] = mtx_df[c] / 60

In [ ]:
if len(cols) > 1:
    mtx_df['fdist'] = mtx_df[cols].min(axis=1)
    mtx_df.drop(cols, axis=1, inplace=True)
else:
    mtx_df.rename(columns={cols[0] : 'fdist'}, inplace=True)

In [ ]:
mtx_df['fdist'] = mtx_df.fdist.fillna(-1)

In [ ]:
idf = idf.merge(mtx_df, how='left', left_index=True, right_index=True)

In [ ]:
df_to_shp(idf, oshp)